In [8]:
import pandas as pd
import numpy as np
from yaml import safe_load
import os
from tqdm import tqdm
import json

In [173]:
filenames = []
for file in os.listdir('data'):
    filenames.append(os.path.join('data',file))

In [174]:
filenames[0:5]

['data\\1001349.yaml',
 'data\\1001351.yaml',
 'data\\1001353.yaml',
 'data\\1004729.yaml',
 'data\\1007655.yaml']

In [175]:
final_df = pd.DataFrame()
counter = 1

In [176]:
for file in tqdm(filenames):
    with open(file, 'r') as f:
        df = pd.json_normalize(safe_load(f))  # Use json.load to read the JSON content
        df['match_id'] = counter
        final_df = pd.concat([final_df, df])  # Append df to final_df
        counter += 1

100%|██████████████████████████████████████████████████████████████████████████████| 1432/1432 [04:18<00:00,  5.54it/s]


In [177]:
final_df.columns

Index(['innings', 'meta.data_version', 'meta.created', 'meta.revision',
       'info.dates', 'info.gender', 'info.match_type',
       'info.outcome.by.wickets', 'info.outcome.winner', 'info.overs',
       'info.player_of_match', 'info.teams', 'info.toss.decision',
       'info.toss.winner', 'info.umpires', 'info.venue', 'match_id',
       'info.city', 'info.outcome.by.runs', 'info.match_type_number',
       'info.neutral_venue', 'info.outcome.method', 'info.outcome.result',
       'info.outcome.eliminator', 'info.supersubs.New Zealand',
       'info.supersubs.South Africa', 'info.bowl_out',
       'info.outcome.bowl_out'],
      dtype='object')

In [178]:
backup = final_df.copy()

In [198]:
final_df.drop(['meta.data_version', 'meta.created', 'meta.revision',
       'info.outcome.by.runs','info.outcome.method', 'info.neutral_venue',
       'info.outcome.eliminator', 'info.match_type_number',
       'info.supersubs.New Zealand', 'info.supersubs.South Africa',
        'info.bowl_out', 'info.outcome.bowl_out',
       'info.outcome.result','info.outcome.by.wickets'],axis=1,inplace=True)

In [199]:
final_df['info.gender'].value_counts()

info.gender
male      966
female    466
Name: count, dtype: int64

In [200]:
final_df = final_df[final_df['info.gender'] == 'male'] #filter out matches played by males only

In [201]:
final_df.drop(['info.gender'],axis=1,inplace = True) #drop gender column

In [202]:
backup2 = final_df.copy()

In [204]:
final_df['info.overs'].unique()

array([20, 50], dtype=int64)

In [205]:
final_df=final_df[final_df['info.overs']== 20] 

In [ ]:
final_df.drop(['info.overs','info.match_type'],axis=1,inplace=True)

In [210]:
import pickle
pickle.dump(final_df,open('dataset_level1.pkl','wb'))

In [2]:
import pickle
matches = pickle.load(open('dataset_level1.pkl','rb'))

In [3]:
matches.iloc[0]['innings'][0]['1st innings']['deliveries']

[{0.1: {'batsman': 'AJ Finch',
   'bowler': 'SL Malinga',
   'non_striker': 'M Klinger',
   'runs': {'batsman': 0, 'extras': 0, 'total': 0}}},
 {0.2: {'batsman': 'AJ Finch',
   'bowler': 'SL Malinga',
   'non_striker': 'M Klinger',
   'runs': {'batsman': 0, 'extras': 0, 'total': 0}}},
 {0.3: {'batsman': 'AJ Finch',
   'bowler': 'SL Malinga',
   'non_striker': 'M Klinger',
   'runs': {'batsman': 1, 'extras': 0, 'total': 1}}},
 {0.4: {'batsman': 'M Klinger',
   'bowler': 'SL Malinga',
   'non_striker': 'AJ Finch',
   'runs': {'batsman': 2, 'extras': 0, 'total': 2}}},
 {0.5: {'batsman': 'M Klinger',
   'bowler': 'SL Malinga',
   'non_striker': 'AJ Finch',
   'runs': {'batsman': 0, 'extras': 0, 'total': 0}}},
 {0.6: {'batsman': 'M Klinger',
   'bowler': 'SL Malinga',
   'non_striker': 'AJ Finch',
   'runs': {'batsman': 3, 'extras': 0, 'total': 3}}},
 {1.1: {'batsman': 'M Klinger',
   'bowler': 'KMDN Kulasekara',
   'non_striker': 'AJ Finch',
   'runs': {'batsman': 0, 'extras': 0, 'total': 

In [4]:
count =1
delivery_df = pd.DataFrame()
for index,row in matches.iterrows():
    if count in [75,108,150,180,268,360,443,458,584,748,982,1052,1111,1226,1345]:
        count+=1
        continue
    count+=1
    ball_of_match=[]
    batsman=[]
    bowler=[]
    runs=[]
    player_of_dismissed=[]
    teams=[]
    batting_team=[]
    match_id=[]
    city=[]
    venue=[]
    for ball in row ['innings'][0]['1st innings']['deliveries']:
        for key in ball.keys():
            match_id.append(count)
            batting_team.append(row['innings'][0]['1st innings']['team'])
            teams.append(row['info.teams'])
            ball_of_match.append(key)
            batsman.append(ball[key]['batsman'])
            bowler.append(ball[key]['bowler'])
            runs.append(ball[key]['runs']['total'])
            city.append(row['info.city'])
            venue.append(row['info.venue'])
            try:
                player_of_dismissed.append(ball[key]['wicket']['player_out'])
            except:
                player_of_dismissed.append('0')
    loop_df = pd.DataFrame({
        'match_id':match_id,
        'teams':teams,
        'batting_team':batting_team,
        'ball':ball_of_match,
        'batsman':batsman,
        'bowler':bowler,
        'runs':runs,
        'player_dismissed':player_of_dismissed,
        'city':city,
        'venue':venue
    })
    delivery_df = pd.concat([delivery_df,loop_df])

NameError: name 'pd' is not defined

In [ ]:
def bowl(row): # function to extract bowling team from playing teams and batting team
    for team in row['teams']:
        if team != row['batting_team']:
            return team

In [ ]:
delivery_df['bowling_team'] =  delivery_df.apply(bowl,axis=1) #applying bowling team column to delivery_df

In [ ]:
delivery_df.drop(['teams'],axis=1,inplace=True)

In [ ]:
delivery_df['batting_team'].unique()

In [ ]:
teams=[
    'India',
    'Australia',
    'New Zealand',
    'Bangladesh',
    'South Africa',
    'West Indies',
    'Afghanistan',
    'Pakistan',
    'Sri Lanka',
    'England'
]

In [ ]:
#filtering out only top ten teams playing cricket as rest of teams have very less matches which may generate bias in our model

delivery_df = delivery_df[delivery_df['batting_team'].isin(teams)]
delivery_df = delivery_df[delivery_df['bowling_team'].isin(teams)]


In [ ]:
output = delivery_df[['match_id','batting_team','bowling_team','ball','runs','player_dismissed','city','venue']]

In [ ]:
import pickle
pickle.dump(output,open('dataset_level2.pkl','wb'))

In [8]:
import pickle
dataset_level2 = pickle.load(open('dataset_level2.pkl','rb'))